## RAG для диалоговых систем
Проект представляет собой ассистента, который отвечает на любые вопросы про жизнь известных личностей. Включает квантизированную модель, векторную базу данных, движок для поиска текста по базе данных, функцию поиска текстов в интернете и поддержку диалогового режима.

In [ ]:
%%capture captured_output
!pip install \
datasets \
autoawq==0.2.6 \
accelerate==0.33.0 \
chromadb==0.5.5 \
langchain==0.2.5 \
langchain-community==0.2.5 \
langchain-chroma \
numpy==1.26.4 \
pandas==2.0.3 \
sentence-transformers==3.0.1 \
torch==2.3.1 \
torchvision==0.18.1 \
transformers==4.44.0 \
tokenizers==0.19.1 \
requests \
beautifulsoup4 \
lxml \
# bitsandbytes==0.42.0 \
# gensim==4.3.2 \
# matplotlib==3.6.2 \
# nltk==3.8.1 \
# peft==0.11.1 \
# scikit-learn==1.3.2 \
# scipy==1.10.1 \
# seqeval==1.2.2 \
# wandb==0.13.10

In [ ]:
import json
import time
import gc
import re
import shutil
import urllib
from typing import List
import numpy as np
from scipy.spatial.distance import euclidean
import torch
from datasets import Dataset
from tqdm.auto import tqdm
from google.colab import drive
import requests
from bs4 import BeautifulSoup
import zipfile
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/кс_dl_nlp/project/'

In [ ]:
with open(path + 'ru_wiki_person.txt', 'r') as f:
    articles = f.read().split('\n\n')

len(articles)

269086

In [ ]:
articles[:2]

['Эльда́р Алекса́ндрович Ряза́нов (18 ноября 1927, Самара, СССР — 30 ноября 2015, Москва, Россия) — советский и российский кинорежиссёр, сценарист, актёр, поэт, драматург, телеведущий, педагог, продюсер; народный артист СССР (1984), лауреат Государственной премии СССР (1977) и Государственной премии РСФСР имени братьев Васильевых (1979).Среди шедевров советской киноклассики, созданных Эльдаром Рязановым, — комедии и мелодрамы «Карнавальная ночь» (1956), «Девушка без адреса» (1957), «Дайте жалобную книгу» (1965), «Берегись автомобиля» (1966), «Старики-разбойники» (1971), «Невероятные приключения итальянцев в России» (1973), «Ирония судьбы, или С лёгким паром» (1976), «Служебный роман» (1977), «Гараж» (1979), «О бедном гусаре замолвите слово» (1980), «Вокзал для двоих» (1982), «Жестокий романс» (1984), «Небеса обетованные» (1991).Рязанов — автор более 200 собственных телевизионных программ, с 1979 по 1985 год вёл телепередачу «Кинопанорама». Автор текста ряда широко популярных романсов, 

In [ ]:
data = {"text": articles}
dataset = Dataset.from_dict(data)

In [ ]:
dataset[0]

{'text': 'Эльда́р Алекса́ндрович Ряза́нов (18 ноября 1927, Самара, СССР — 30 ноября 2015, Москва, Россия) — советский и российский кинорежиссёр, сценарист, актёр, поэт, драматург, телеведущий, педагог, продюсер; народный артист СССР (1984), лауреат Государственной премии СССР (1977) и Государственной премии РСФСР имени братьев Васильевых (1979).Среди шедевров советской киноклассики, созданных Эльдаром Рязановым, — комедии и мелодрамы «Карнавальная ночь» (1956), «Девушка без адреса» (1957), «Дайте жалобную книгу» (1965), «Берегись автомобиля» (1966), «Старики-разбойники» (1971), «Невероятные приключения итальянцев в России» (1973), «Ирония судьбы, или С лёгким паром» (1976), «Служебный роман» (1977), «Гараж» (1979), «О бедном гусаре замолвите слово» (1980), «Вокзал для двоих» (1982), «Жестокий романс» (1984), «Небеса обетованные» (1991).Рязанов — автор более 200 собственных телевизионных программ, с 1979 по 1985 год вёл телепередачу «Кинопанорама». Автор текста ряда широко популярных ро

### Соберем векторную базу данных.
База знаний состоит из первых абзацев русскоязычных статей из википедии про различных людей.



In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [ ]:
db = Chroma(
    collection_name="ru_wiki_person",
    embedding_function=embedding_function,
    persist_directory="chroma_db",
)

In [ ]:
def add_batch_to_db(batch):
    documents = []
    for item in batch:
        text_chunks = text_splitter.split_text(item)
        batch_documents = [Document(page_content=text) for text in text_chunks]
        documents.extend(batch_documents)

    db.add_documents(
        documents=documents,
    )

def save_db_to_drive():
    shutil.make_archive("chroma_db", 'zip', "chroma_db")
    shutil.copy("chroma_db.zip", path)

In [ ]:
batch_size = 1000

for i in tqdm(range(0, len(dataset), batch_size), desc="Загрузка батчей в БД"):
    add_batch_to_db(dataset[i:i + batch_size]['text'])

save_db_to_drive()

Загрузка батчей в БД:   0%|          | 0/270 [00:00<?, ?it/s]

### Протестируем работу модели

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_function = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [ ]:
model_name = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

tokenizer = AutoTokenizer.from_pretrained(model_name)

quantization_config = AwqConfig(bits=4, fuse_max_seq_len=2048, do_fuse=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            device_map="auto",
            quantization_config=quantization_config
        )

Проверим, насколько хорошо модель отвечает на вопрос без использования RAG.

In [ ]:
def llm_answer(query, model, tokenizer):
    messages = [
        {"role": "user", "content": f"Запрос: {query}"}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    ).to("cuda")
    outputs = model.generate(
        **inputs,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.3,
        max_new_tokens=512
    )[0]
    response = tokenizer.decode(outputs[inputs['input_ids'].shape[1]:], skip_special_tokens=True).replace('\n\n', '')

    return response

In [ ]:
query = "Кто такой Павел Воля?"

llm_answer(query, model, tokenizer)

'Павел Воля - польский актер, комик и телеведущий. Он известен как один из ведущих актеров польской комедии.Павел Воля родился 23 июня 1970 года в городе Кракове, Польша. Он получил образование в Краковском университете, где изучал актерское искусство.Павел Воля начал свою карьеру в 1990-х годах, снимаясь в польских телесериалах и фильмах. Он быстро стал известным актером и комиком, и его работы часто вызывали смех и восторг у зрителей.Павел Воля также известен как телеведущий. Он часто вёл различные телепередачи, включая комедийные шоу и ток-шоу.В 2010-х годах Павел Воля стал известен как один из ведущих актеров польской комедии. Он часто снимался в польских телесериалах и фильмах, и его работы часто вызывали смех и восторг у зрителей.Павел Воля также известен как телеведущий. Он часто вёл различные телепередачи, включая комедийные шоу и ток-шоу.В 2010-х годах Павел Воля стал известен как один из ведущих актеров польской комедии. Он часто снимался в польских телесериалах и фильмах, и 

Видим, что знаний модели не хватает. Встречается неверная иноформация.

Проверим, насколько хорошо ищутся похожие по смыслу тексты.

In [ ]:
path_to_db = '/content/drive/MyDrive/кс_dl_nlp/project/chroma_db.zip'
with zipfile.ZipFile(path_to_db, 'r') as zip_ref:
    zip_ref.extractall('/content/chroma_db')

In [ ]:
db = Chroma(
    collection_name="ru_wiki_person",
    embedding_function=embedding_function,
    persist_directory="chroma_db",
)

In [ ]:
query = "Кто такой Павел Воля?"

results = db.similarity_search_with_relevance_scores(query)
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
sorted_results

[(Document(page_content='Васи́лий Алекса́ндрович Во́лга (род. 5 марта 1968, Северодонецк, УССР, СССР) — украинский оппозиционный политический деятель, лидер политической партии «Союз Левых Сил» с 2007 года.Народный депутат Верховной рады Украины V созыва (2006—2007).'),
  0.7903295649753691),
 (Document(page_content='Ра́фал Во́льский (; 10 ноября 1992, Козенице) — польский футболист, атакующий полузащитник клуба «Висла» (Плоцк) .'),
  0.7442050865628351),
 (Document(page_content='}Вальдема́р Павля́к, (; род. 5 сентября 1959, селение Модель, Варшавское воеводство, ПНР) — польский политик и общественный деятель. Депутат Сейма ПНР X созыва (1989-1991), Председатель Совета министров в 1992 и 1993—1995 годах, Министр экономики Польши в двух правительствах Дональда Туска (2007-2012), депутат Сейма I, II, III, IV, V, VI и VII созывов с 1991 по 2015, член Сената XI созыва с 2023 года. Дважды председатель Польской народной партии (1991-1997 и 2005-2012).'),
  0.740217783959462),
 (Document(page

In [ ]:
def semantic_search(client, query, k=3):
    results = client.similarity_search_with_relevance_scores(query, k=k)
    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

    return [res[0].page_content for res in sorted_results]

In [ ]:
def llm_answer(query, context_list, model, tokenizer):
    context = "\n".join(context_list)
    system_message = (
        "Ты полезный ассистент.\n"
        "Пожалуйста, дай ответ на запрос пользователя, используя только данную тебе информацию в контексте.\n"
        "Дай ответ только на основе, предоставленного тебе контекста.\n"
        "Убедись, что твой ответ точен и не содержит никакой другой информации."
        f"Контекст: {context}\n"
    )
    messages = [
        {"role": "user", "content": system_message},
        {"role": "user", "content": f"Запрос: {query}"},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    ).to("cuda")
    outputs = model.generate(
        **inputs,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.3,
        max_new_tokens=512
    )[0]
    response = tokenizer.decode(outputs[inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    return response

In [ ]:
query = "Кто такой Павел Воля?"
context = semantic_search(db, query, k=3)

llm_answer(query, context, model, tokenizer)

'Павел Воля (род. 5 сентября 1959) — польский политик и общественный деятель.'

Модель отвечает на вопрос на основе поданного контекста, но информация в бд может быть по разным причинам совершенно не подходящей.

Добавим функцию поиска в интернете, так как в базе данных может не быть релевантной информации

In [ ]:
def get_relevant_chunks(text, query_emb, similarity_threshold):
    doc = Document(page_content=text)
    splitted_docs = text_splitter.split_documents([doc])
    score_fn = db._select_relevance_score_fn()

    relevant_chunks = []
    for doc in splitted_docs:
        doc_embedding = db._embedding_function.embed_query(doc.page_content)
        score = score_fn(euclidean(doc_embedding, query_emb)**2)

        if score > similarity_threshold:
            relevant_chunks.append(doc.page_content)

    return relevant_chunks

In [ ]:
def google_search(query, k, similarity_threshold):
    print('Ищу в гугле...')

    escaped_query = urllib.parse.quote_plus(query)
    google_url = f"https://www.google.com/search?q={escaped_query}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3538.102 Safari/537.36"
    }

    response = requests.get(google_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    anchors = soup.find_all("a", href=True)

    links = []
    for a in anchors:
        anchor_links = list(filter(lambda l: l.startswith("url=http"), a["href"].split("&")))
        links.extend(link.split("url=")[-1] for link in anchor_links if len(link) > 0)

    query_emb = db._embedding_function.embed_query(query)
    out_documents = []
    for link in links:
        link = link.replace('%25', '%').replace('%3F', '?').replace('%3D', '=')
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        text = re.sub(r'[\n]+', '\n', soup.get_text()).replace('\xa0', ' ')
        out_documents.extend(get_relevant_chunks(text, query_emb, similarity_threshold))

        if len(out_documents) >= k:
            break

    return out_documents[:k]

In [ ]:
def search(query, k, similarity_threshold):
    docs_scores = db.similarity_search_with_relevance_scores(query, k=k)

    good_documents = []
    for doc, score in docs_scores:
        if score > similarity_threshold:
            good_documents.append(doc.page_content)

    if len(good_documents) == k:
        return good_documents

    good_documents += google_search(query, k - len(good_documents), similarity_threshold)

    return good_documents

In [ ]:
query = "Кто такой Павел Воля, и где он родился?"
context = search(query, k=4, similarity_threshold=0.80)
print(context)
llm_answer(query, context, model, tokenizer)

Ищу в гугле...
['Воля, Павел Алексеевич — Википедия\nВоля, Павел Алексеевич\nМатериал из Википедии — свободной энциклопедии\nСтабильная версия, проверенная 14 июня 2024.\n20 правок ожидают проверки.\nПерейти к навигации\nПерейти к поиску\nВ Википедии есть статьи о других людях с такой фамилией, см. Воля.\nПавел Воля Павел Воля на книжном фестивале «Красная площадь — 2022» \nИмя при рождении\nПавел Алексеевич Воля\nДата рождения\n14 марта 1979(1979-03-14) (45 лет)\nМесто рождения\nПенза, СССР\nГражданство\n Россия\nПрофессия\nактёр, певец, поэт, телеведущий, шоумен, комик\nНаправление\nпоп и стенд-ап\nIMDb\nID 2910093\npavelvolya.com\n Медиафайлы на Викискладе\nПа́вел Алексе́евич Во́ля[1] (род. 14 марта 1979, Пенза) — российский эстрадный артист разговорного жанра, телеведущий, поэт, киноактёр, певец, ведущий Comedy Club.', 'Павел Воля биография, фото, карьера, личная жизнь\nПерейти к основному контенту\n            Телеканал\n                    \n            Pro\n                    \

'Павел Воля — российский комик, актёр, телеведущий, сценарист, певец, актёр дубляжа, продюсер, шоумен, резидент и ведущий Comedy Club.\n\nОн родился 14 марта 1979 года в Пензе.'

### RAG
Видим, что ответ стал намного лучше.
Теперь соберем все в одну систему RAG и добавим поддержку диалога.

In [ ]:
class Search:
    """
    Этот класс умеет искать релевантные тексты из базы данных по запросу
    Если релевантных текстов нет в базе данных, то класс будет искать их в интернете
    """
    def __init__(self, db, similarity_threshold: float = 0.72, k: int = 3):
        """
        db: langchain Chroma
        similarity_threshold: порог, по которому оценивается релевантность запросу
        k: число извлекаемых текстов
        """
        self.db = db
        self.score_function = self.db._select_relevance_score_fn()

        self.similarity_threshold = similarity_threshold
        self.k = k

        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    def search(self, query: str):
        docs_scores = self.db.similarity_search_with_relevance_scores(query, k=self.k)

        good_documents = []
        for doc, score in docs_scores:
            if score > self.similarity_threshold:
                good_documents.append(doc.page_content)

        if len(good_documents) == self.k:
            return good_documents

        good_documents += self.google_search(query, k=self.k - len(good_documents))

        return good_documents

    def google_search(self, query, k):
        # print('Ищу в гугле...')

        escaped_query = urllib.parse.quote_plus(query)
        google_url = f"https://www.google.com/search?q={escaped_query}"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3538.102 Safari/537.36"
        }

        response = requests.get(google_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        anchors = soup.find_all("a", href=True)

        links = []
        for a in anchors:
            anchor_links = list(filter(lambda l: l.startswith("url=http"), a["href"].split("&")))
            links.extend(link.split("url=")[-1] for link in anchor_links if len(link) > 0)

        query_emb = self.db._embedding_function.embed_query(query)
        out_documents = []
        for link in links:
            link = link.replace('%25', '%').replace('%3F', '?').replace('%3D', '=')
            response = requests.get(link, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")
            text = re.sub(r'[\n]+', '\n', soup.get_text()).replace('\xa0', ' ')
            out_documents.extend(self.get_relevant_chunks(text, query_emb))

            if len(out_documents) >= k:
                break

        return out_documents[:k]

    def get_relevant_chunks(self, text: str, query_emb: List[float]) -> List[str]:
        doc = Document(page_content=text)
        splitted_docs = self.text_splitter.split_documents([doc])

        relevant_chunks = []
        for doc in splitted_docs:
            doc_embedding = self.db._embedding_function.embed_query(doc.page_content)
            score = self.score_function(euclidean(doc_embedding, query_emb)**2)

            if score > self.similarity_threshold:
                relevant_chunks.append(doc.page_content)

        return relevant_chunks

In [ ]:
class RAG:
    def __init__(self, search_engine):
        self.search_engine = search_engine
        self.history = []

        model_name = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # AWQ работает только с GPU!
        quantization_config = AwqConfig(bits=4, fuse_max_seq_len=2048, do_fuse=True)
        self.gen_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            device_map={"": "cuda"},
            quantization_config=quantization_config
        )

    def _summarize_user_intent(self, query: str) -> str:
        """
        Используется для поддержки истории диалога.
        Составляет точный запрос пользователя, используя новый запрос и всю предыдущую историю.
        """
        if len(self.history) == 0:
            return query

        chat_history_str = ""
        for entry in self.history:
            chat_history_str += f"{entry[0]}: {entry[1]}\n"

        system_message = (
            "Ты ассистент, который читает запись разговора между искусственным интеллектом и пользователем. "
            "Имея историю разговора и новый запрос пользователя, переформулируй запрос с учетом истории так, "
            "чтобы он имел однозначный ответ. Замени все местоимения и общие слова на конкретные сущности. "
            "Не пиши ответ на этот запрос, а также вообще ничего дополнительного, кроме переформулированного запроса.\n"
            f"История разговора: ```{chat_history_str}```\n\n"
            f"Запрос пользователя: ```{query}```\n"
        )

        messages = [
            {"role": "user", "content": system_message},
        ]

        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            return_dict=True,
        ).to("cuda")

        outputs = self.gen_model.generate(
            **inputs,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.3,
            max_new_tokens=512
        )[0]
        result = self.tokenizer.decode(outputs[inputs['input_ids'].shape[1]:], skip_special_tokens=True)

        return result

    def _rag(self, context_list: list[str], query: str) -> str:
        """
        Генерируем ответ на запрос используя контекст.
        """
        self.history.append(('Пользователь', query))

        context = "\n".join(context_list)
        system_message = (
            "Ты полезный ассистент.\n"
            "Пожалуйста, дай ответ на запрос пользователя, используя только данную тебе информацию в контексте.\n"
            "Убедись, что твой ответ точен и не содержит никакой другой информации."
            f"Контекст: ```{context}```\n"
        )

        messages = [
            {"role": "user", "content": system_message},
            {"role": "user", "content": f"Запрос: {query}"},
        ]

        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            return_dict=True,
        ).to("cuda")

        outputs = self.gen_model.generate(
            **inputs,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.3,
            max_new_tokens=512
        )[0]
        response = self.tokenizer.decode(outputs[inputs['input_ids'].shape[1]:], skip_special_tokens=True)

        self.history.append(('ИИ', response))

        return response

    def answer(self, query: str) -> str:
        """
        Генерирует ответ на запрос query
        """
        user_intent = self._summarize_user_intent(query)
        context_list = self.search_engine.search(user_intent)
        response = self._rag(context_list, user_intent)

        return response

    def clear_history(self):
        self.history = []

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")
db = Chroma(persist_directory="chroma_db", embedding_function=embedding_function)
search_engine = Search(db, similarity_threshold=0.72, k=3)
rag = RAG(search_engine)

<ipython-input-13-87cc23af1730>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/63.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def interactive_rag_session(rag_instance):
    print("Введите ваш вопрос (или 'exit' для завершения, 'clear' для очистки истории):")
    while True:
        query = input("> ")
        if query.lower() == "exit":
            break
        elif query.lower() == "clear":
            rag_instance.clear_history()
            print("История очищена.\n")
            continue
        response = rag_instance.answer(query)
        print(f"Ответ: {response}\n")

Тестируем работу RAG

In [ ]:
interactive_rag_session(rag)

Введите ваш вопрос (или 'exit' для завершения, 'clear' для очистки истории):
> clear
История очищена.

> Кто сыграл Форреста Гампа в одноимённом фильме?
Ответ: В роли Форреста Гампа в одноименном фильме снялся актер Том Хэнкс.

> В каком году вышел этот фильм?
Ответ: Ответ: Фильм "Форрест Гамп" вышел в 1994 году.

> exit
